## Imports y configuraciones

In [ ]:
import os
import shutil
import cv2
import kagglehub
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from collections import Counter
from imblearn.over_sampling import SMOTE
from skimage.feature import graycomatrix, graycoprops, hog, local_binary_pattern
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import learning_curve, train_test_split
from sklearn.svm import SVC
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, Input, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import to_categorical

## Funciones auxiliares

In [ ]:
# Funciones auxiliares

def plot_distribution(y, title):
    """
    Muestra la distribución de clases en un gráfico de barras.

    Parámetros:
    - y: etiquetas de las clases
    - title: título del gráfico
    """
    counter = Counter(y)
    print(f"Distribución de clases ({title}):", counter)

    plt.figure(figsize=(10, 5))
    sns.barplot(x=list(counter.keys()), y=list(counter.values()))
    plt.xlabel("Clase")
    plt.ylabel("Cantidad de muestras")
    plt.title(title)
    plt.xticks(rotation=0)
    plt.show()


def plot_confusion_matrix(y_true, y_pred, feature_name, label_map):
    """
    Genera y muestra una matriz de confusión con etiquetas legibles.

    Parámetros:
    - y_true: etiquetas reales
    - y_pred: etiquetas predichas
    - feature_name: nombre del extractor de características usado
    - label_map: diccionario de etiquetas de las clases
    - title_suffix: texto adicional para el título (ej. "antes de SMOTE" o "después de SMOTE")
    """
    cm = confusion_matrix(y_true, y_pred)

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_map.values(), yticklabels=label_map.values())
    plt.xlabel("Predicción")
    plt.ylabel("Real")
    plt.title(f"Matriz de Confusión - {feature_name}")
    plt.show()


def train_model(X, y):
    """
    Entrena un modelo SVM con los datos proporcionados.

    Parámetros:
    - X: características de entrada
    - y: etiquetas

    Retorna:
    - X_train, X_test, y_train, y_test, y_pred
    """
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    svm = SVC(kernel='linear', class_weight='balanced')
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test)
    return X_train, X_test, y_train, y_test, y_pred


# Ref. https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html
def evaluate_model(y_test, y_pred, feature_name, X_train, y_train):
    """
    Evalúa el modelo entrenado y muestra el reporte de clasificación junto con la curva de aprendizaje.

    Parámetros:
    - y_test: etiquetas reales
    - y_pred: etiquetas predichas
    - feature_name: nombre del extractor de características usado
    - X_train: datos de entrenamiento
    - y_train: etiquetas de entrenamiento
    """
    print(f"Resultados para {feature_name}:")
    print(classification_report(y_test, y_pred))

    train_sizes, train_scores, test_scores = learning_curve(
        # Analiza cómo varía la precisión al entrenar con diferentes cantidades de datos
        SVC(kernel='linear', class_weight='balanced'),
        X_train, y_train,
        cv=5,
        scoring='accuracy',
        train_sizes=np.linspace(0.1, 1.0, 3),  # Reduce aún más el número de pruebas → [0.1, 0.55, 1.0]
        n_jobs=-1  # Usa todos los núcleos disponibles de la CPU para paralelizar el cálculo
    )

    plt.figure(figsize=(8, 6))
    plt.plot(train_sizes, train_scores.mean(axis=1), label="Entrenamiento")
    plt.plot(train_sizes, test_scores.mean(axis=1), label="Validación")
    plt.xlabel("Cantidad de Datos")
    plt.ylabel("Precisión")
    plt.legend()
    plt.title(f"Curva de Aprendizaje - {feature_name}")
    plt.show()


# Crear el modelo ViT usando Keras
def create_vit_model():
    """Modelo Vision Transformer simplificado en TensorFlow/Keras"""
    inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

    # Convertir imagen en parches y proyectar
    x = layers.Conv2D(64, (PATCH_SIZE, PATCH_SIZE), strides=(PATCH_SIZE, PATCH_SIZE), padding="valid")(inputs)
    x = layers.Reshape((-1, x.shape[-1]))(x)

    # Aplicar Multi-Head Attention
    x = layers.MultiHeadAttention(num_heads=8, key_dim=64)(x, x)
    x = layers.LayerNormalization()(x)

    # Aplanar y añadir capas densas
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation="relu")(x)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

    return keras.Model(inputs, outputs)

## Carga del dataset

In [ ]:
# Ruta del dataset
dataset_final_path = "data/EuroSAT/"
train_csv_path = "data/EuroSAT/train.csv"

# Descarga el dataset si no existe en la carpeta data
if not os.path.exists(dataset_final_path):
    print("Descargando el dataset...")
    download_path = kagglehub.dataset_download("apollo2506/eurosat-dataset")

    # Buscar la carpeta correcta dentro de la descarga
    possible_folders = ["EuroSAT", "EuroSATallBands"]
    original_path = None

    for folder in possible_folders:
        candidate_path = os.path.join(download_path, folder)
        if os.path.exists(candidate_path):
            original_path = candidate_path
            break

    # Si no se encontró la carpeta esperada, lanzar un error
    if original_path is None:
        print(f"Error: No se encontró ninguna de las carpetas {possible_folders} dentro de {download_path}.")
        exit()

    # Asegurar que la carpeta de destino exista
    os.makedirs(os.path.dirname(dataset_final_path), exist_ok=True)

    # Mover el dataset a la carpeta `data/EuroSAT/`
    shutil.move(original_path, dataset_final_path)
    print(f"Dataset movido a: {dataset_final_path}")
else:
    print("El dataset ya está descargado. No es necesario volver a descargarlo")

In [ ]:
# Cargar el archivo CSV con la lista de imágenes a usar
df = pd.read_csv(train_csv_path)

In [ ]:
# Reduce el dataset seleccionando aleatoriamente un 50% de las filas
# reduction_factor = 0.5  # Proporción del dataset a conservar
# df = df.sample(frac=reduction_factor, random_state=42).reset_index(drop=True)
# print(f"Dataset reducido al {reduction_factor * 100}% del tamaño original")

In [ ]:
# Ver las primeras filas del CSV
display(df.head())

In [ ]:
# Extraer nombres de archivo y etiquetas
image_filenames = df["Filename"].tolist()
image_labels = df["Label"].tolist()

# Crear un diccionario para mapear etiquetas numéricas a nombres de clase
label_map = dict(zip(df["ClassName"].unique(), df["Label"].unique()))

print("Etiquetas:", label_map)

## Extracción de características, entrenamiento, evaluación y visualización de resultados de los modelos tradicionales

In [ ]:
# Extracción de características Haralick, HOG y LBP de las imágenes

# Ref. https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.graycomatrix
# Ref. https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.graycoprops
# Ref. https://medium.com/top-python-libraries/12-examples-of-image-texture-analysis-in-python-6cf7c179ada7
# Ref. https://medium.com/@girishajmera/feature-extraction-of-images-using-glcm-gray-level-cooccurrence-matrix-e4bda8729498
# Ref. https://medium.com/swlh/histogram-of-oriented-gradients-hog-for-multiclass-image-classification-and-image-recommendation-cf0ea2caaae8


# Parámetros para calcular la matriz de co-ocurrencia de niveles de gris (GLCM)
distances = [1]  # Distancia de 1 píxel entre los valores a comparar
angles = [0]  # Ángulo de 0 grados (horizontal) para evaluar la textura

X_haralick, X_hog, X_lbp, X_combined, y = [], [], [], [], []

# Recorrer las imágenes listadas en train.csv
for img_filename, label in zip(image_filenames, image_labels):
    img_path = os.path.join(dataset_final_path, img_filename)

    # Cargar la imagen en escala de grises
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    # Si la imagen no se puede leer, se ignora
    if img is None:
        continue

    # Calcula la matriz de co-ocurrencia de niveles de gris (GLCM)
    glcm = graycomatrix(img, distances, angles, symmetric=True, normed=True)

    # Extracción de características de Haralick
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    asm = graycoprops(glcm, 'ASM')[0, 0]
    haralick_features = [contrast, dissimilarity, homogeneity, energy, correlation, asm]

    # Extracción de características usando el histograma de gradientes orientados (HOG)
    hog_features = hog(img, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)

    # Extracción de características usando Local Binary Patterns (LBP)
    lbp_features = local_binary_pattern(img, P=8, R=1).flatten()

    # Almacenar las características en listas separadas
    X_haralick.append(haralick_features)
    X_hog.append(list(hog_features))
    X_lbp.append(list(lbp_features))
    X_combined.append(haralick_features + list(hog_features) + list(lbp_features))
    y.append(label)

print("Extracción de características completada")

In [ ]:
# Convierte los datos en arrays
X_haralick = np.array(X_haralick)
X_hog = np.array(X_hog)
X_lbp = np.array(X_lbp)
X_combined = np.array(X_combined)
y = np.array(y)

In [ ]:
# Visualizar la distribución de clases antes de aplicar SMOTE
plot_distribution(y, "Distribución de Clases Antes de SMOTE")

In [ ]:
# Balanceo de clases con SMOTE
smote = SMOTE(random_state=42)
X_haralick_resampled, y_resampled = smote.fit_resample(X_haralick, y)
X_hog_resampled, _ = smote.fit_resample(X_hog, y)
X_lbp_resampled, _ = smote.fit_resample(X_lbp, y)
X_combined_resampled, _ = smote.fit_resample(X_combined, y)

In [ ]:
# Visualizar la distribución de clases después de aplicar SMOTE
plot_distribution(y_resampled, "Distribución de Clases Después de SMOTE")

In [ ]:
# Entrenamiento y evaluación con Haralick (GLCM)
X_train_haralick, X_test_haralick, y_train_haralick, y_test_haralick, y_pred_haralick = train_model(
    X_haralick_resampled, y_resampled)

evaluate_model(y_test_haralick, y_pred_haralick, "Haralick (GLCM)", X_train_haralick, y_train_haralick)

In [ ]:
# Entrenamiento y evaluación con HOG
X_train_hog, X_test_hog, y_train_hog, y_test_hog, y_pred_hog = train_model(
    X_hog_resampled, y_resampled)

evaluate_model(y_test_hog, y_pred_hog, "HOG", X_train_hog, y_train_hog)

In [ ]:
# Entrenamiento y evaluación con LBP
X_train_lbp, X_test_lbp, y_train_lbp, y_test_lbp, y_pred_lbp = train_model(
    X_lbp_resampled, y_resampled)

evaluate_model(y_test_lbp, y_pred_lbp, "LBP", X_train_lbp, y_train_lbp)

In [ ]:
# Entrenamiento y evaluación combinando todos los extractores de características
X_train_combined, X_test_combined, y_train_combined, y_test_combined, y_pred_combined = train_model(
    X_combined_resampled, y_resampled)

evaluate_model(y_test_combined, y_pred_combined, "Haralick + HOG + LBP", X_train_combined, y_train_combined)

In [ ]:
# Mostrar la matriz de confusión para Haralick (GLCM)
plot_confusion_matrix(y_test_haralick, y_pred_haralick, "Haralick (GLCM)", label_map)

In [ ]:
# Mostrar la matriz de confusión para HOG
plot_confusion_matrix(y_test_hog, y_pred_hog, "HOG", label_map)

In [ ]:
# Mostrar la matriz de confusión para LBP
plot_confusion_matrix(y_test_lbp, y_pred_lbp, "LBP", label_map)

In [ ]:
# Mostrar la matriz de confusión para la combinación de Haralick + HOG + LBP
plot_confusion_matrix(y_test_combined, y_pred_combined, "Haralick + HOG + LBP", label_map)

## Redes convolucionales (CNNs)

In [ ]:
# Tamaño de las imágenes para la CNN
IMG_SIZE = 64
NUM_CLASSES = len(label_map)

# Cargar imágenes y etiquetas en formato adecuado
X = []
y = []

for img_filename, label in zip(image_filenames, image_labels):
    img_path = os.path.join(dataset_final_path, img_filename)
    img = cv2.imread(img_path)  # Mantener las imágenes en color (RGB)
    if img is None:
        continue
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  # Redimensionar
    img = img / 255.0  # Normalizar entre 0 y 1
    X.append(img)
    y.append(label)

# Convertir a arrays de NumPy
X = np.array(X)
y = np.array(y)

# One-hot encoding de las etiquetas
y = to_categorical(y, num_classes=NUM_CLASSES)

print("Carga de imágenes completada")

In [ ]:
# Definir la arquitectura de la CNN
model = Sequential([
    Input(shape=(IMG_SIZE, IMG_SIZE, 3)),  # Definir la forma de entrada aquí

    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Regularización para evitar overfitting
    Dense(NUM_CLASSES, activation='softmax')  # Clasificación multiclase
])

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Mostrar la arquitectura del modelo
model.summary()

In [ ]:
# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar la CNN
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Precisión en test: {test_acc:.4f}")

In [ ]:
# Obtener los valores de accuracy
epochs = range(1, len(history.history['accuracy']) + 1)  # Números de época

# Crear la gráfica con puntos y líneas
plt.figure(figsize=(8, 6))
plt.plot(epochs, history.history['accuracy'], marker='o', linestyle='-', label='Entrenamiento')  # Línea + puntos
plt.plot(epochs, history.history['val_accuracy'], marker='s', linestyle='-',
         label='Validación')  # Cuadrados para validación

# Agregar etiquetas y leyenda
plt.xlabel("Épocas")
plt.ylabel("Precisión")
plt.title("Curva de aprendizaje - CNN con Puntos")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.5)  # Cuadrícula más suave

# Mostrar la gráfica
plt.show()

## ViT (Vision Transformers)

In [ ]:
# Parámetros del modelo
IMG_SIZE = 64  # Tamaño de imagen de entrada
PATCH_SIZE = 8  # Tamaño de los parches
NUM_CLASSES = 10  # Número de clases (ajustar según el dataset)

# Crear y compilar el modelo
vit_model = create_vit_model()
vit_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Mostrar resumen del modelo
vit_model.summary()

In [ ]:
# Entrenar el modelo ViT
history_vit = vit_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

print("✅ Entrenamiento de ViT completado")

In [ ]:
# Evaluar el modelo ViT en test
test_loss_vit, test_acc_vit = vit_model.evaluate(X_test, y_test)
print(f"✅ Precisión en test - ViT: {test_acc_vit:.4f}")

In [ ]:
# Obtener los valores de accuracy para ViT
epochs = range(1, len(history_vit.history['accuracy']) + 1)  # Números de época

# Crear la gráfica con puntos y líneas
plt.figure(figsize=(8, 6))
plt.plot(epochs, history_vit.history['accuracy'], marker='o', linestyle='-', label='Entrenamiento')  # Línea + puntos
plt.plot(epochs, history_vit.history['val_accuracy'], marker='s', linestyle='-',
         label='Validación')  # Cuadrados para validación

# Agregar etiquetas y leyenda
plt.xlabel("Épocas")
plt.ylabel("Precisión")
plt.title("Curva de aprendizaje - ViT con Puntos")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.5)  # Cuadrícula más suave

# Mostrar la gráfica
plt.show()

## ResNet50


In [ ]:
# Definir la arquitectura de ResNet50 con pesos preentrenados en ImageNet
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Congelar las capas del modelo base para reutilizar sus características
base_model.trainable = False

# Agregar capas personalizadas para ajustar el modelo a nuestro dataset
x = Flatten()(base_model.output)  # Aplanar salida de ResNet
x = Dense(128, activation="relu")(x)  # Capa densa intermedia
x = Dense(NUM_CLASSES, activation="softmax")(x)  # Capa de salida para clasificación

# Definir el modelo final
resnet_model = Model(inputs=base_model.input, outputs=x)

# Compilar el modelo
resnet_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Mostrar la arquitectura
resnet_model.summary()

In [ ]:
history_resnet = resnet_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
test_loss, test_acc = resnet_model.evaluate(X_test, y_test)
print(f"Precisión en test (ResNet50): {test_acc:.4f}")

In [ ]:
# Obtener los valores de accuracy
epochs = range(1, len(history_resnet.history['accuracy']) + 1)

# Crear la gráfica con puntos y líneas
plt.figure(figsize=(8, 6))
plt.plot(epochs, history_resnet.history['accuracy'], marker='o', linestyle='-', label='Entrenamiento')
plt.plot(epochs, history_resnet.history['val_accuracy'], marker='s', linestyle='-', label='Validación')

# Agregar etiquetas y leyenda
plt.xlabel("Épocas")
plt.ylabel("Precisión")
plt.title("Curva de aprendizaje - ResNet50 con Puntos")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.5)

# Mostrar la gráfica
plt.show()